Big-endian is an order in which the "big end" (most significant value in the sequence) is stored first (at the lowest storage address). 

Little-endian is an order in which the "little end" (least significant value in the sequence) is stored first. 

For example, in a big-endian computer, the two bytes required for the hexadecimal number 4F52 would be stored as 4F52 in storage (if 4F is stored at storage address 1000, for example, 52 will be at address 1001). In a little-endian system, it would be stored as 524F (52 at address 1000, 4F at 1001).

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <unistd.h>
#include <sys/types.h>
#include "asm.h"

#define BUFSIZE 32
#define FLAGSIZE 64

void win() {
  char buf[FLAGSIZE];
  FILE *f = fopen("flag.txt","r");
  if (f == NULL) {
    printf("Flag File is Missing. Problem is Misconfigured, please contact an Admin if you are running this on the shell server.\n");
    exit(0);
  }

  fgets(buf,FLAGSIZE,f);
  printf(buf);
}

void vuln(){
  char buf[BUFSIZE];
  gets(buf);

  printf("Okay, time to return... Fingers Crossed... Jumping to 0x%x\n", get_return_address());
}

int main(int argc, char **argv){

  setvbuf(stdout, NULL, _IONBF, 0);
  
  gid_t gid = getegid();
  setresgid(gid, gid, gid);

  puts("Please enter your string: ");
  vuln();
  return 0;
}

setvbuf(stdout, NULL, _IONBF, 0);

This first function call will set stdout's buffering type to unbuffered.

This means that anything written to stdout will imedialty be written to stdout. 
This bypasses the usual line buffering that is the default behavior.

gid_t gid = getegid();<br>
setresgid(gid, gid, gid);

getegid() gets the effective group id of the calling proccess.

setresgid() sets the real GID, effective GID, and saved set-group-ID 
of the calling process (andalways  modifies  the  filesystem  GID  
to be the same as the effective GID), with the same  restrictions 
for unprivileged processes.


We'll need to know some stuff about assembly.

The __lea__ (load effective address) instruction is used to put a memory address into the destination.

The __push__ instruction is used to push values on the stack.

__eax__ is a 32-bit general-purpose register with two common uses: to store the return value of a function and as a special register for certain calculations. It is technically a volatile register, since the value isn't preserved. Instead, its value is set to the return value of a function before a function returns.


Memory Layout of a C program

Vissualization: http://i.stack.imgur.com/1Yz9K.gif

![alt text](http://i.stack.imgur.com/1Yz9K.gif "BIG STACK")

    Command line arguments and environment variables: The arguments passed to a program before running and the environment variables are stored in this section.

    Stack: This is the place where all the function parameters, return addresses and the local variables of the function are stored. It’s a LIFO structure. It grows downward in memory(from higher address space to lower address space) as new function calls are made. We will examine the stack in more detail later.

    Heap: All the dynamically allocated memory resides here. Whenever we use malloc to get memory dynamically, it is allocated from the heap. The heap grows upwards in memory(from lower to higher memory addresses) as more and more memory is required.

    Uninitialized data(Bss Segment): All the uninitialized data is stored here. This consists of all global and static variables which are not initialized by the programmer. The kernel initializes them to arithmetic 0 by default.

    Initialized data(Data Segment): All the initialized data is stored here. This constists of all global and static variables which are initialised by the programmer.

    Text: This is the section where the executable code is stored. The loader loads instructions from here and executes them. It is often read only.


Registers to know



__%eip:__ The Instruction pointer register. It stores the address of the next instruction to be executed. After every instruction execution it’s value is incremented depending upon the size of an instrution.

__%esp:__ The Stack pointer register. It stores the address of the top of the stack. This is the address of the last element on the stack. The stack grows downward in memory(from higher address values to lower address values). So the %esp points to the value in stack at the lowest memory address.

__%ebp:__ The Base pointer register. The %ebp register usually set to %esp at the start of the function. This is done to keep tab of function parameters and local variables. Local variables are accessed by subtracting offsets from %ebp and function parameters are accessed by adding offsets to it.
